In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import dlib
import argparse
import operator
import cv2
import glob
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adagrad

In [11]:
dataset = '/Users/dustint/Documents/Face Charisma Project/SuccessVsNonsuccessProcessed'
data_dir = pathlib.Path(dataset)

In [12]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [13]:
#setup training 
img_height,img_width=224,224
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 1563 files belonging to 2 classes.
Using 312 files for validation.


In [14]:
#setup validation
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset='validation',
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1563 files belonging to 2 classes.
Using 312 files for validation.


In [6]:
#show class names
class_names = train_ds.class_names
print(class_names)

['Nonsuccess', 'Success']


In [15]:
#model structure
resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(224,224,3),
                                                  pooling='max',
                                                  classes=2,
                                                  weights='imagenet')
for layer in pretrained_model.layers:
  layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(50, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [ ]:
#*1 and sigmoid => a single binary classi... one value return which is prob(True)
#2 and sigmoid => 2 multi-output binary classi.... two values of prob(True) for each of the channels/outputs
#    => is(normal) and is(celebrity)
#Both cases should use binary_cross_entropy
    
#*2 and softmax (categorical_crossentropy)

In [16]:
#model summary
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 50)                102450    
                                                                 
 dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 23,690,264
Trainable params: 102,552
Non-trainable params: 23,587,712
_________________________________________________________________


In [17]:
#model define
resnet_model.compile(optimizer=Adagrad(learning_rate=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_ds.class_names

In [18]:
#model training
epochs=15
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/15
10/10 [==============================] - 23s 2s/step - loss: 2.8515 - accuracy: 0.5321 - val_loss: 2.3652 - val_accuracy: 0.5737
Epoch 2/15
10/10 [==============================] - 21s 2s/step - loss: 1.1895 - accuracy: 0.6154 - val_loss: 0.7042 - val_accuracy: 0.7051
Epoch 3/15
10/10 [==============================] - 20s 2s/step - loss: 0.7286 - accuracy: 0.7244 - val_loss: 0.6441 - val_accuracy: 0.7244
Epoch 4/15
10/10 [==============================] - 20s 2s/step - loss: 0.5272 - accuracy: 0.7821 - val_loss: 0.4267 - val_accuracy: 0.8109
Epoch 5/15
10/10 [==============================] - 21s 2s/step - loss: 0.4310 - accuracy: 0.8077 - val_loss: 0.3412 - val_accuracy: 0.8526
Epoch 6/15
10/10 [==============================] - 20s 2s/step - loss: 0.3589 - accuracy: 0.8429 - val_loss: 0.2938 - val_accuracy: 0.8718
Epoch 7/15
10/10 [==============================] - 20s 2s/step - loss: 0.2973 - accuracy: 0.8910 - val_loss: 0.2939 - val_accuracy: 0.8750
Epoch 8/15
10/10 [==

In [19]:
resnet_model.save('/Users/dustint/Documents/Face Charisma Project/SuccessResnet50.h5')